In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


In [4]:
train_df = pd.read_csv('Resources/2019loans.csv')
test_df = pd.read_csv('Resources/2020Q1loans.csv')

In [5]:
train_df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,n,28.42,0.0,0.0,...,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N,high_risk
12176,15000.0,0.1774,540.34,RENT,50000.0,Verified,n,23.43,4.0,0.0,...,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N,high_risk
12177,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,n,28.80,0.0,1.0,...,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N,high_risk
12178,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,n,11.44,0.0,0.0,...,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N,high_risk


In [39]:
# Convert categorical data to numeric and separate target feature for training data
y_train=train_df['target'].values
X_train=train_df.drop('target', axis=1)
X_train_dummies=pd.get_dummies(X_train)
y_train_label=LabelEncoder().fit_transform(y_train)
target_names=['low risk','high risk']
len(X_train_dummies.columns)

92

In [26]:
# Convert categorical data to numeric and separate target feature for testing data
y_test=test_df['target'].values
X_test=test_df.drop('target', axis=1)
X_test_dummies=pd.get_dummies(X_test)
y_test_label=LabelEncoder().fit_transform(y_test)
len(X_test_dummies.columns)

91

In [27]:
# add missing dummy variables to testing set
for x in X_train_dummies.columns:
    if x not in X_test_dummies.columns:
        X_test_dummies[x]=0
len(X_test_dummies.columns)

92

## Consider the models
My prediction is that the RandomForest model will perform better than the LogisticRegression model.  I believe this since typically randomforest is better when there is lots of catagorical data and possible outliers like this data set does. 

In [48]:
classifier = LogisticRegression()
classifier.fit(X_train_dummies, y_train_label)
print(f"Training Data Score: {classifier.score(X_train_dummies, y_train_label)}")
print(f"Testing Data Score: {classifier.score(X_test_dummies, y_test_label)}")



Training Data Score: 0.653448275862069
Testing Data Score: 0.5091450446618461
              precision    recall  f1-score   support

    low risk       0.52      0.30      0.38      2351
   high risk       0.51      0.72      0.59      2351

    accuracy                           0.51      4702
   macro avg       0.51      0.51      0.49      4702
weighted avg       0.51      0.51      0.49      4702



/Users/gregoryfujitani/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
# Train a Random Forest Classifier model and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_dummies, y_train_label)

print(f'Training Score: {clf.score(X_train_dummies, y_train_label)}')
print(f'Testing Score: {clf.score(X_test_dummies, y_test_label)}')

Training Score: 1.0
Testing Score: 0.646958740961293


In [51]:
# Scale the data
scaler = StandardScaler().fit(X_train_dummies)
X_train_scaled = scaler.transform(X_train_dummies)
X_test_scaled = scaler.transform(X_test_dummies)

In [53]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression()
classifier.fit(X_train_scaled, y_train_label)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train_label)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test_label)}")

Training Data Score: 0.710919540229885
Testing Data Score: 0.7598894087622289


/Users/gregoryfujitani/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train_label)

print(f"Training Data Score: {clf.score(X_train_scaled, y_train_label)}")
print(f"Testing Data Score: {clf.score(X_test_scaled, y_test_label)}")

Training Data Score: 1.0
Testing Data Score: 0.6480221182475542


## Final Analysis

Comparing the final scores, the LogisticRegression improved after scaling going from 0.509 to 0.760.  RandomForest score ended up being roughly the same with the non-scaled score being 0.647 and the scaled 0.648.  It makes sense that the LogisticRegression improved with scaled data as the data had values that were in the tens of thousands and some that were decimals. RandomForest did not really improved and this expected since it is using particioning instead of distance based algorithims.
In the end with scaled data, my prediction was wrong since the LogisticRegression performed better than the RandomForest.